## 配置环境变量

这里主要添加了本地的MindSponge路径，方便本地的MindSponge和打包安装的MindSponge的协同使用，另外配置了一些简单的环境变量。

In [ ]:
import sys
sys.path.append('../../../src')
import os
os.environ['GLOG_v'] = '4'
os.environ['MS_JIT_MODULES'] = 'sponge'

## 导入相关模块

导入mindspore和mindsponge两个模块，基本上就可以实现大部分计算的功能。如果需要使用一些第三方库来进行前处理或者后处理，也可以自行添加。

In [1]:
from mindspore import context
from mindspore.nn import Adam
from sponge import Sponge, Molecule, ForceField, set_global_units
from sponge.callback import WriteH5MD, RunInfo

## 设置全局变量

一个是全局单位，另一个是选择MindSpore的静态图/动态图构建模式，以及运行的平台，和设备编号等。

In [2]:
set_global_units('nm', 'kj/mol')
# context.set_context(mode=context.GRAPH_MODE, device_target='Ascend', device_id=0)
context.set_context(mode=context.GRAPH_MODE, device_id=0)

## 定义系统

Molecule是一个MindSponge中较为基础的分子类型，所有的体系都可以用Molecule这一基础类型来进行定义。如果是MindSponge已经支持的一部分模型，我们可以直接加载这些模型来定义一个分子系统，然后使用reduplicate和copy等方法对系统进行扩展。定义完一个系统之后，可以查看系统的一些属性，比如原子名称和键连关系等。

In [3]:
system = Molecule(template='water.spce.yaml')
system.reduplicate([0.3, 0, 0])
new_sys = system.copy([0, 0, -0.3])
system.append(new_sys)

Molecule<>

In [4]:
print('The atom names in the system are: ', system.atom_name)

The atom names in the system is:  [['O' 'H1' 'H2' 'O' 'H1' 'H2' 'O' 'H1' 'H2' 'O' 'H1' 'H2']]


In [5]:
print('Bonds in the system are: ', system.bonds)

Bonds in the system are:  [[[ 0  1]
  [ 0  2]
  [ 3  4]
  [ 3  5]
  [ 6  7]
  [ 6  8]
  [ 9 10]
  [ 9 11]]]


## 定义力场

这里是非常关键的一步，我们需要通过给定的分子系统，和给定的力场参数，来进行建模。进而可以使用MindSpore内置的优化器，或者我们自定义的优化器/积分器来进行迭代。最后使用Sponge类将系统、力场和迭代器封装起来，就完成了一个分子迭代器的定义。而在迭代过程中的每一步，都是可以微分的，也都可以追溯其单点能。

In [6]:
potential = ForceField(system, parameters='SPCE')
opt = Adam(system.trainable_params(), 1e-3)
mini = Sponge(system, potential, opt)

In [7]:
print('The single point energy of given system is: ', mini.calc_energy())

The single point energy of given system is:  [[110.0423]]


In [9]:
energy_names = mini.energy_names
energies = mini.calc_energies()
for i, name in enumerate(energy_names):
    print('The {} energy is: {}'.format(name, energies[0][i]))

The bond_energy energy is: 3.8302694e-11
The angle_energy energy is: 2.7213787e-11
The coulomb_energy energy is: 105.32028
The lj_energy energy is: 4.7220154


## 回调方法

MindSponge不仅仅支持基础的RunInfo，将每一步的能量输出到屏幕上，还支持了hdf5格式的轨迹文件输出。这种hdf5格式的输出，既可以使用silx-view这一工具来查看，也可以在构建安装好相关的VMD插件之后，使用VMD来进行动态可视化。

In [10]:
run_info = RunInfo(50)
cb_h5md = WriteH5MD(system, 'tutorial_c01.h5md', save_freq=50, write_velocity=True, write_force=True)
mini.run(500, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 09:37:20
Warning! The optimizer "Adam<>" does not has the attribute "velocity".
[MindSPONGE] Step: 0, E_pot: 110.0423
[MindSPONGE] Step: 50, E_pot: 46.733253
[MindSPONGE] Step: 100, E_pot: -40.781235
[MindSPONGE] Step: 150, E_pot: -86.66056
[MindSPONGE] Step: 200, E_pot: -148.19458
[MindSPONGE] Step: 250, E_pot: -148.3973
[MindSPONGE] Step: 300, E_pot: -148.40488
[MindSPONGE] Step: 350, E_pot: -148.40501
[MindSPONGE] Step: 400, E_pot: -148.40486
[MindSPONGE] Step: 450, E_pot: -148.40503
[MindSPONGE] Finished simulation at 2023-08-15 09:37:26
[MindSPONGE] Simulation time: 5.62 seconds.
--------------------------------------------------------------------------------
